In [1]:
import os
import pandas as pd
import re
import unicodedata

# Ruta de la carpeta donde están los archivos
folder_path = "Base de Datos - ONGD"

# Lista para almacenar los DataFrames
dataframes = []

# Mapeo de columnas para normalización
column_mapping = {
    "DENOMINACIÓN": "INSTITUCIÓN",  
    "Entidad": "INSTITUCIÓN",
    "Institución": "INSTITUCIÓN",
    "SIGLAS": "SIGLAS",             
    "DIRECCIÓN": "DOMICILIO",        
    "DOMICILIO": "DOMICILIO",
    "DEPARTAMENTO": "DEPARTAMENTO",
    "PROVINCIA": "PROVINCIA",
    "DISTRITO": "DISTRITO",
    "ÁREA": "ÁREA"
}

# Función para extraer semestre desde el nombre del archivo
def obtener_semestre(file_name):
    file_name = file_name.replace(".xlsx", "").replace(".xls", "").strip()
    
    match = re.search(r"(\d{4})_(06|12)", file_name)
    
    if match:
        año = match.group(1)
        mes = match.group(2)  

        semestre_ordenable = int(f"{año}{mes}")  
        semestre_legible = f"{'I' if mes == '06' else 'II'} semestre {año}"
        
        return semestre_legible, semestre_ordenable
    
    else:
        return "Desconocido", 0

# Función para normalizar nombres de instituciones
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return "DESCONOCIDO"
    nombre = str(nombre).upper().strip()
    nombre = unicodedata.normalize('NFKD', nombre).encode('ASCII', 'ignore').decode("utf-8")  
    nombre = re.sub(r'[^\w\s]', '', nombre)  
    nombre = re.sub(r'(_X000D_|X000D)', '', nombre)  # Eliminar basura de caracteres ocultos
    return nombre

# Recorrer los archivos en la carpeta
for file in os.listdir(folder_path):
    if file.startswith("~$"):  
        continue
    
    if file.endswith(".xls") or file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)

        # Extraer semestre
        semestre, semestre_ordenable = obtener_semestre(file)

        # Determinar el engine
        engine = "xlrd" if file.endswith(".xls") else "openpyxl"

        try:
            df = pd.read_excel(file_path, engine=engine, dtype=str)
        except PermissionError:
            print(f"⚠️ No se pudo leer el archivo {file} (posiblemente esté abierto).")
            continue  

        df.rename(columns={col: column_mapping[col] for col in df.columns if col in column_mapping}, inplace=True)

        # Normalizar nombres de instituciones
        if "INSTITUCIÓN" in df.columns:
            df["INSTITUCIÓN"] = df["INSTITUCIÓN"].apply(normalizar_nombre)

        for col in ["INSTITUCIÓN", "SIGLAS", "DOMICILIO", "DEPARTAMENTO", "PROVINCIA", "DISTRITO", "ÁREA"]:
            if col not in df.columns:
                df[col] = None

        df["SEMESTRE"] = semestre
        df["SEMESTRE_ORDEN"] = semestre_ordenable

        dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
df_final = pd.concat(dataframes, ignore_index=True)

# Ordenar por institución y semestre (antiguos primero)
df_final = df_final.sort_values(by=["INSTITUCIÓN", "SEMESTRE_ORDEN"])

# Llenar datos faltantes con la información más reciente dentro de cada institución
df_final = df_final.groupby("INSTITUCIÓN").apply(lambda group: group.ffill().bfill()).reset_index(drop=True)

# Detectar ONGs con interrupciones de más de 3 años y separarlas
df_final["DIFERENCIA_AÑOS"] = df_final.groupby("INSTITUCIÓN")["SEMESTRE_ORDEN"].diff()
df_final["SEPARAR"] = df_final["DIFERENCIA_AÑOS"] > 300  # Si hay una diferencia mayor a 3 años (300 en formato YYYYMM)

df_final["SEPARADOR_TEMPORAL"] = df_final.groupby("INSTITUCIÓN")["SEPARAR"].cumsum()
df_final["INSTITUCIÓN_UNIFICADA"] = df_final["INSTITUCIÓN"] + " - " + df_final["SEPARADOR_TEMPORAL"].astype(str)

# Agrupar nuevamente considerando interrupciones
df_final = df_final.groupby("INSTITUCIÓN_UNIFICADA", as_index=False).agg({
    "SEMESTRE_ORDEN": ["min", "max"],  
    "SIGLAS": "last",         
    "DOMICILIO": "last",      
    "DEPARTAMENTO": "last",   
    "PROVINCIA": "last",      
    "DISTRITO": "last",       
    "ÁREA": "last"            
})

# Renombrar columnas correctamente
df_final.columns = ["INSTITUCIÓN", "PRIMER SEMESTRE_ORDEN", "ÚLTIMO SEMESTRE_ORDEN",
                    "SIGLAS", "DOMICILIO", "DEPARTAMENTO", "PROVINCIA", "DISTRITO", "ÁREA"]

# Convertir SEMESTRE_ORDEN a su formato legible
df_final["PRIMER SEMESTRE"] = df_final["PRIMER SEMESTRE_ORDEN"].astype(str).str[:4] + " " + df_final["PRIMER SEMESTRE_ORDEN"].astype(str).str[4:].replace({"06": "I semestre", "12": "II semestre"})
df_final["ÚLTIMO SEMESTRE"] = df_final["ÚLTIMO SEMESTRE_ORDEN"].astype(str).str[:4] + " " + df_final["ÚLTIMO SEMESTRE_ORDEN"].astype(str).str[4:].replace({"06": "I semestre", "12": "II semestre"})

# Eliminar las columnas temporales
df_final.drop(columns=["PRIMER SEMESTRE_ORDEN", "ÚLTIMO SEMESTRE_ORDEN"], inplace=True)

# Guardar el resultado final en un nuevo archivo Excel
output_path = os.path.join(folder_path, "Base_Datos_Unificada.xlsx")
df_final.to_excel(output_path, index=False)

print("✅ ¡Base de datos combinada y normalizada correctamente!")


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6148\3968674709.py:96: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final = df_final.groupby("INSTITUCIÓN").apply(lambda group: group.ffill().bfill()).reset_index(drop=True)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6148\3968674709.py:96: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = df_final.groupby("INSTITUCIÓN").apply(lambda group: group.ffill().bfill()).reset_index(drop=True)


✅ ¡Base de datos combinada y normalizada correctamente!
